## **Importing the Data**

In [ ]:
import pandas as pd
df=pd.read_csv('/content/spam.csv',encoding='latin1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


### **PrEporcessing THe Data**

In [ ]:
model=df[['v1','v2']]

In [ ]:
model.rename(columns={'v1':'classification','v2':'message'},inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
model.head()

,classification,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
from sklearn.preprocessing import LabelEncoder
lr =LabelEncoder()
model.classification=lr.fit_transform(model['classification'])

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [ ]:
model.head()

,classification,message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
validation=model.sample(frac=1,random_state=42)

In [ ]:
round(sum([len(i.split()) for i in train_s]))/len(train_s)

15.544087951536909

In [ ]:
from sklearn.model_selection import train_test_split
train_s,val_s,train_l,val_l=train_test_split(validation['message'].to_numpy(),
                                             validation['classification'].to_numpy(),
                                             test_size=0.2,
                                             random_state=42)

##creating an embedding layers and vectorisation layer

In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
text=TextVectorization(max_tokens=10000, standardize='lower_and_strip_punctuation', 
                  split='whitespace',  output_mode='int',
                 output_sequence_length=16 )

In [ ]:
text.adapt(train_s)

In [ ]:
text(['U dun say so early hor'])

<tf.Tensor: shape=(1, 16), dtype=int64, numpy=
array([[   7,  231,  143,   25,  357, 3162,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0]])>

In [ ]:
from tensorflow.keras import layers
embedding=layers.Embedding(input_dim=10000, output_dim=128, input_length=15)

In [ ]:
embedding(text(['U dun say so early hor']))

<tf.Tensor: shape=(1, 16, 128), dtype=float32, numpy=
array([[[ 0.01461962, -0.04034798,  0.00816289, ...,  0.0495217 ,
          0.01984784,  0.01650982],
        [-0.03301577,  0.01200842,  0.02763175, ..., -0.01119313,
          0.00454601, -0.00124257],
        [-0.00825678,  0.01423505,  0.03996028, ..., -0.00251818,
          0.03166192,  0.01081327],
        ...,
        [ 0.04471819,  0.03558356, -0.01816682, ...,  0.04342682,
         -0.00059314, -0.01581129],
        [ 0.04471819,  0.03558356, -0.01816682, ...,  0.04342682,
         -0.00059314, -0.01581129],
        [ 0.04471819,  0.03558356, -0.01816682, ...,  0.04342682,
         -0.00059314, -0.01581129]]], dtype=float32)>

## a simple model with embedding and vectosisation indduced with global average pooling

In [ ]:
import tensorflow as tf

inputs=layers.Input(shape=(1,),dtype='string')
x=text(inputs)
x=embedding(x)
x=layers.GlobalAveragePooling1D()(x)
output=layers.Dense(1,activation='sigmoid')(x)

model=tf.keras.Model(inputs,output)

In [ ]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)

model.fit(train_s,
          train_l,
          epochs=5,
          validation_data=(val_s,val_l))

Epoch 1/5
140/140 [==============================] - 4s 8ms/step - loss: 0.4168 - accuracy: 0.8851 - val_loss: 0.2399 - val_accuracy: 0.8960
Epoch 2/5
140/140 [==============================] - 1s 6ms/step - loss: 0.1576 - accuracy: 0.9502 - val_loss: 0.1442 - val_accuracy: 0.9507
Epoch 3/5
140/140 [==============================] - 1s 6ms/step - loss: 0.0951 - accuracy: 0.9744 - val_loss: 0.1099 - val_accuracy: 0.9668
Epoch 4/5
140/140 [==============================] - 1s 7ms/step - loss: 0.0681 - accuracy: 0.9838 - val_loss: 0.0935 - val_accuracy: 0.9686
Epoch 5/5
140/140 [==============================] - 1s 7ms/step - loss: 0.0518 - accuracy: 0.9874 - val_loss: 0.0833 - val_accuracy: 0.9704


In [ ]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 16)                0         
_________________________________________________________________
embedding (Embedding)        (None, 16, 128)           1280000   
_________________________________________________________________
global_average_pooling1d_7 ( (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


## **a simple LSTM stacked model**

In [ ]:
inputs=layers.Input(shape=(1,),dtype='string')
x=text(inputs)
x=embedding(x)
x=layers.LSTM(64,return_sequences=True)(x)
x=layers.LSTM(32)(x)
x=layers.Dense(32,activation='relu')(x)
output=layers.Dense(1,activation='sigmoid')(x)

model_1=tf.keras.Model(inputs,output)
model_1.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 16)                0         
_________________________________________________________________
embedding (Embedding)        (None, 16, 128)           1280000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 16, 64)            49408     
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense_9 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 33  

In [ ]:
model_1.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)

model_1.fit(train_s,
          train_l,
          epochs=5,
          validation_data=(val_s,val_l))

Epoch 1/5
140/140 [==============================] - 7s 23ms/step - loss: 0.0669 - accuracy: 0.9904 - val_loss: 0.0895 - val_accuracy: 0.9731
Epoch 2/5
140/140 [==============================] - 2s 15ms/step - loss: 0.0112 - accuracy: 0.9980 - val_loss: 0.0884 - val_accuracy: 0.9776
Epoch 3/5
140/140 [==============================] - 2s 16ms/step - loss: 0.0038 - accuracy: 0.9989 - val_loss: 0.1017 - val_accuracy: 0.9749
Epoch 4/5
140/140 [==============================] - 2s 15ms/step - loss: 0.0010 - accuracy: 0.9996 - val_loss: 0.1606 - val_accuracy: 0.9731
Epoch 5/5
140/140 [==============================] - 2s 15ms/step - loss: 6.4187e-04 - accuracy: 1.0000 - val_loss: 0.1399 - val_accuracy: 0.9740
